<a href="https://colab.research.google.com/github/h1-ti/DFDNet/blob/master/DFDNET_FACE_ENHANCER_VIA_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **実行前の準備**



1.   GOOGOLE DRIVE を開き左上の『＋新規』→　フォルダ　→　faceset フォルダを作成
2.   ローカル（自分のパソコン）の 1)Faceset/CATEGORYフォルダ を開き、超解像したいSRCをzipで圧縮する。例）1)Faceset/hogezaka/hoge -> 1)Faceset/hogezaka/hoge.zip
3.1のfacesetフォルダに2のhoge.zipをアップロード（複数可）



# **実行**



* このページの上にある　ランタイム（左から５番目）　→　すべてのセルを実行

## ※注意  **1.MOUNT GOOGLE DRIVE** のところで入力があります

### 詳しく

* GOOGLE DRIVEのマウントが完了したら終わるまで待つだけです


* 完了すると先ほどのfacesetフォルダにhoge_dfdnet.zipができるのでドライブからダウンロードしてください


* ※srcが多すぎると最後まで完了せずに接続が切れるかもしれません。時間をおいてやり直してください。完了したものはスキップされるようになっております

* 実行中に新しいsrcのzipをアップすることも可能です


* ※また、2.INSTALL and GET MODEL WEIGHTでダウンロードがうまくいかず止まっている場合はやり直してください

**CHECK VIDEO CARD'S SPEC**


速さ比較

 P100(?f/s) > T4(6f/s) > K80(3f/s)

In [ ]:
!nvidia-smi

Tue Oct 25 07:26:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **1.MOUNT GOOGLE DRIVE**

### ①Go to this URL in a browser:の右のリンクからGOOGLEにログイン
### ②Enter your authorization code:の下に①で取得したコードをペーストしてGOOGLE DRIVE をマウント



In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# **2.INSTALL and GET MODEL WEIGHT**

### install DFDNet

In [ ]:
%cd /content
!git clone --branch master https://github.com/h1-ti/DFDNet.git

### get weights

In [ ]:
!pip install dominate

from IPython.display import clear_output
from google.colab import files
from PIL import Image
import matplotlib.pyplot as plt
import warnings
import os

warnings.filterwarnings("ignore")
%matplotlib inline


os.chdir('/content/DFDNet/')
!pip install -r requirements.txt

link_prefix = 'https://github.com/rocketsvm/DFDNet/releases/download/20200827/'
wget_options = '-q --show-progress --progress=bar:noscroll --no-check-certificate -r'
!wget $wget_options $link_prefix'latest_net_G.pth' -O 'latest_net_G.pth'
!wget $wget_options $link_prefix'left_eye_32_center.npy' -O 'left_eye_32_center.npy'
!wget $wget_options $link_prefix'left_eye_64_center.npy' -O 'left_eye_64_center.npy'
!wget $wget_options $link_prefix'left_eye_128_center.npy' -O 'left_eye_128_center.npy'
!wget $wget_options $link_prefix'left_eye_256_center.npy' -O 'left_eye_256_center.npy'
!wget $wget_options $link_prefix'right_eye_32_center.npy' -O 'right_eye_32_center.npy'
!wget $wget_options $link_prefix'right_eye_64_center.npy' -O 'right_eye_64_center.npy'
!wget $wget_options $link_prefix'right_eye_128_center.npy' -O 'right_eye_128_center.npy'
!wget $wget_options $link_prefix'right_eye_256_center.npy' -O 'right_eye_256_center.npy'
!wget $wget_options $link_prefix'mouth_32_center.npy' -O 'mouth_32_center.npy'
!wget $wget_options $link_prefix'mouth_64_center.npy' -O 'mouth_64_center.npy'
!wget $wget_options $link_prefix'mouth_128_center.npy' -O 'mouth_128_center.npy'
!wget $wget_options $link_prefix'mouth_256_center.npy' -O 'mouth_256_center.npy'
!wget $wget_options $link_prefix'nose_32_center.npy' -O 'nose_32_center.npy'
!wget $wget_options $link_prefix'nose_64_center.npy' -O 'nose_64_center.npy'
!wget $wget_options $link_prefix'nose_128_center.npy' -O 'nose_128_center.npy'
!wget $wget_options $link_prefix'nose_256_center.npy' -O 'nose_256_center.npy'
!wget $wget_options $link_prefix'vgg19.pth' -O 'vgg19.pth'
!mkdir -p /content/DFDNet/checkpoints/facefh_dictionary
!mv latest_net_G.pth /content/DFDNet/checkpoints/facefh_dictionary
!mkdir -p /content/DFDNet/DictionaryCenter512
!mv *_center.npy /content/DFDNet/DictionaryCenter512
!mkdir -p /content/DFDNet/weights
!mv vgg19.pth /content/DFDNet/weights

# **3.FACE ENHANCE PROCESS**


In [ ]:
%cd /content/DFDNet

/content


In [ ]:
#@title FACE ENHANCE PROCESS

import sys
import os, os.path as osp
import argparse
import zipfile
import glob
import cv2
import shutil
import numpy as np
from PIL import Image

import torchvision.transforms as transforms
import torch
from tqdm import tqdm

from util import util
import models
from models import create_model
import data
from data.image_folder import make_dataset
from options.test_options import TestOptions
from DFLIMG.DFLPNG import DFLPNG
from DFLIMG.DFLJPG import DFLJPG

root = "/content/DFDNET"

OUT_RES = 512
    
faceset_dir = "/content/drive/MyDrive/faceset"
indir = osp.join(root, "temp_input")
os.makedirs(indir, exist_ok=True)

class TestOptions2(TestOptions):
    def __init__(self) -> None:
        super().__init__()
    
    # override
    def gather_options(self):
        # initialize parser with basic options
        if not self.initialized:
            parser = argparse.ArgumentParser(
                formatter_class=argparse.ArgumentDefaultsHelpFormatter)
            parser = self.initialize(parser)
        
        opt, _ = parser.parse_known_args()
        # modify model-related parser options
        model_name = opt.model
        model_option_setter = models.get_option_setter(model_name)
        parser = model_option_setter(parser, self.isTrain)

        opt, _ = parser.parse_known_args()  # parse again with the new defaults

        # modify dataset-related parser options
        dataset_name = opt.dataset_mode

        dataset_option_setter = data.get_option_setter(dataset_name)
        parser = dataset_option_setter(parser, self.isTrain)

        self.parser = parser
        opt, _ = parser.parse_known_args()
        return opt

def get_part_location(lmrks):
    Landmarks = []
    
    Landmarks = np.array(lmrks)
    
    Map_LE = list(np.hstack((range(17,22), range(36,42))))
    Map_RE = list(np.hstack((range(22,27), range(42,48))))
    Map_NO = list(range(29,36))
    Map_MO = list(range(48,68))
    try:
        #left eye
        Mean_LE = np.mean(Landmarks[Map_LE],0)
        L_LE = np.max((np.max(np.max(Landmarks[Map_LE],0) - np.min(Landmarks[Map_LE],0))/2,16))
        Location_LE = np.hstack((Mean_LE - L_LE + 1, Mean_LE + L_LE)).astype(int)
        #right eye
        Mean_RE = np.mean(Landmarks[Map_RE],0)
        L_RE = np.max((np.max(np.max(Landmarks[Map_RE],0) - np.min(Landmarks[Map_RE],0))/2,16))
        Location_RE = np.hstack((Mean_RE - L_RE + 1, Mean_RE + L_RE)).astype(int)
        #nose
        Mean_NO = np.mean(Landmarks[Map_NO],0)
        L_NO = np.max((np.max(np.max(Landmarks[Map_NO],0) - np.min(Landmarks[Map_NO],0))/2,16))
        Location_NO = np.hstack((Mean_NO - L_NO + 1, Mean_NO + L_NO)).astype(int)
        #mouth
        Mean_MO = np.mean(Landmarks[Map_MO],0)
        L_MO = np.max((np.max(np.max(Landmarks[Map_MO],0) - np.min(Landmarks[Map_MO],0))/2,16))
        Location_MO = np.hstack((Mean_MO - L_MO + 1, Mean_MO + L_MO)).astype(int)
    except:
        return 0
    return torch.from_numpy(Location_LE).unsqueeze(0), torch.from_numpy(Location_RE).unsqueeze(0), torch.from_numpy(Location_NO).unsqueeze(0), torch.from_numpy(Location_MO).unsqueeze(0)

opt = TestOptions2().parse()

opt.gpu_ids = [0] # gpu id. if use cpu, set opt.gpu_ids = []
jpeg_quality = opt.jpeg_quality
model = create_model(opt)
model.setup(opt)

done = []

while True:
    src_names = [osp.splitext(d)[0] for d in os.listdir(faceset_dir)
             if osp.splitext(d)[1] == ".zip" and "_dfdnet" not in d and "_superres" not in d
             and not osp.exists(osp.join(faceset_dir, osp.splitext(d)[0]+"_dfdnet.zip"))
             and osp.splitext(d)[0] not in done]
    if len(src_names) == 0:
        break
    print(src_names)
    
    for id, src in enumerate(src_names):
        print("{} / {} [{}]".format(id+1, len(src_names), src))
        zip_path = osp.join(faceset_dir, src+".zip")
        print("Extracting [{}] ...".format(zip_path))
        with zipfile.ZipFile(zip_path) as z:
            z.extractall(indir)

        files = sorted(glob.glob(osp.join(indir, src, '*.*g')))

        outdir = osp.join(root, src+"_dfdnet", src+"_dfdnet")
        os.makedirs(outdir, exist_ok=True)

        for n, file in enumerate(tqdm(files[:], total=len(files), desc="Enhancing faces ...")):
            filename = osp.basename(file)
            ext = osp.splitext(file)[1]
            if ext == ".jpg":
                dflimg = DFLJPG.load(file)
            elif ext == ".png":
                dflimg = DFLPNG.load(file)
            else:
                continue

            if not dflimg:
                continue

            Landmarks = dflimg.get_landmarks()
            # InputData = dflimg.dfl_dict
        
            # scale landmarks and xseg polys to output image size
            scale_factor = OUT_RES / dflimg.get_shape()[0]
            # print('Scale factor: {}'.format(scale_factor))
            Landmarks = Landmarks * scale_factor
            
            Part_locations = get_part_location(Landmarks)
            A = Image.open(file).convert('RGB')
            
            if Part_locations == 0:
                print('\t################ Error in landmarks, continue...')
                continue
            C = A
            A = A.resize((OUT_RES, OUT_RES), Image.BICUBIC)
            A = transforms.ToTensor()(A) 
            C = transforms.ToTensor()(C)
            A = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(A)
            C = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(C)
            
            data = {'A':A.unsqueeze(0), 'C':C.unsqueeze(0), 'A_paths': file,'Part_locations': Part_locations}

            model.set_input(data)
            try:
                model.test()
                visuals = model.get_current_visuals()
                
                im_data = visuals['fake_A']
                im = util.tensor2im(im_data)
                image_pil = Image.fromarray(im)
                if ext == ".jpg":
                    image_pil.save(osp.join(outdir, filename), quality=jpeg_quality)
                else:
                    image_pil.save(osp.join(outdir, filename))
            
            except Exception as e:
                print('\t################ Error enhancing {}'.format(str(e)))
                continue

            if ext == ".jpg":
                _fanseg_mask = dflimg.dfl_dict.get('fanseg_mask', None)
                if _fanseg_mask is not None:
                    ret, buf = cv2.imencode( '.jpg', _fanseg_mask, [int(cv2.IMWRITE_JPEG_QUALITY), 85] )

                    if ret and len(buf) < 64000:
                        _fanseg_mask = buf
                    else:
                        io.log_err("Unable to encode fanseg_mask for %s" % (filename) )
                        _fanseg_mask = None

                _xseg_mask = dflimg.dfl_dict.get('xseg_mask', None)
                if _xseg_mask is not None:
                    ret, buf = cv2.imencode( '.jpg', _xseg_mask, [int(cv2.IMWRITE_JPEG_QUALITY), 85] )

                    if ret and len(buf) < 64000:
                        _xseg_mask = buf
                    else:
                        io.log_err("Unable to encode xseg_mask for %s" % (filename) )
                        _xseg_mask = None

                DFLJPG.embed_dfldict (osp.join(outdir, filename), 
                                        {'face_type': dflimg.get_face_type(),
                                            'landmarks': dflimg.get_landmarks(),
                                            'ie_polys' : dflimg.get_ie_polys(),
                                            'source_filename': dflimg.get_source_filename(),
                                            'source_rect': dflimg.get_source_rect(),
                                            'source_landmarks': dflimg.get_source_landmarks(),
                                            'image_to_face_mat': dflimg.get_image_to_face_mat(),
                                            'fanseg_mask' : _fanseg_mask,
                                            'xseg_mask' : _xseg_mask,
                                            'eyebrows_expand_mod' : None,
                                            'relighted' : None,
                                            "histgram" : None,
                                            "recognition" : dflimg.get_recognition(),
                                        })
            elif ext == ".png":
                dflimg = DFLPNG.embed_dfldict (osp.join(outdir, filename), 
                                        {'face_type': dflimg.get_face_type(),
                                            'landmarks': Landmarks,
                                            'ie_polys' : dflimg.get_ie_polys(),
                                            'source_filename': dflimg.get_source_filename(),
                                            'source_rect': dflimg.get_source_rect(),
                                            'source_landmarks': dflimg.get_source_landmarks(),
                                            'image_to_face_mat': dflimg.get_image_to_face_mat(),
                                            'fanseg_mask' : dflimg.dfl_dict.get ('fanseg_mask', None),
                                            'xseg_mask' : dflimg.dfl_dict.get('xseg_mask', None),
                                            'eyebrows_expand_mod' : None,
                                            'relighted' : None,
                                            "histgram" : None,
                                            "recognition" : dflimg.get_recognition(),
                                        })


        
        shutil.make_archive(src+"_dfdnet", 'zip', src+"_dfdnet")
        if osp.exists(osp.join(faceset_dir, src+"_dfdnet.zip")):
            os.remove(osp.join(faceset_dir, src+"_dfdnet.zip"))
        shutil.move(src+"_dfdnet.zip", faceset_dir)
        shutil.rmtree(src+"_dfdnet")
        print("Successfully completed {}".format(src+"_dfdnet"))
        done.append(src)

print(done)